In [3]:
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
import os


In [4]:
def generate_report_cards(file_path):
    try:
        df = pd.read_excel(file_path)

        required_columns = {'Student ID', 'Name', 'Subject', 'Score'}
        if not required_columns.issubset(df.columns):
            raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")

        student_summary = (
            df.groupby(['Student ID', 'Name'])
            .agg(Total_Score=('Score', 'sum'), Average_Score=('Score', 'mean'))
            .reset_index()
        )
        output_dir = "report_cards"
        os.makedirs(output_dir, exist_ok=True)
        for _, student in student_summary.iterrows():
            student_id = student['Student ID']
            student_name = student['Name']
            total_score = student['Total_Score']
            average_score = student['Average_Score']

            student_scores = df[df['Student ID'] == student_id][['Subject', 'Score']]
            
            pdf_path = os.path.join(output_dir, f"report_card_{student_id}.pdf")
            generate_pdf(pdf_path, student_name, total_score, average_score, student_scores)
        print(f"Report cards generated successfully in the '{output_dir}' directory.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [5]:
def generate_pdf(pdf_path, student_name, total_score, average_score, student_scores):
    styles = getSampleStyleSheet()

    pdf = SimpleDocTemplate(pdf_path, pagesize=letter)
    elements = []

    elements.append(Paragraph(f"Report Card", styles['Title']))
    elements.append(Paragraph(f"Name: {student_name}", styles['Normal']))
    elements.append(Paragraph(f"Total Score: {total_score}", styles['Normal']))
    elements.append(Paragraph(f"Average Score: {average_score:.2f}", styles['Normal']))
    elements.append(Spacer(1, 12))

    table_data = [['Subject', 'Score']] + student_scores.values.tolist()
    table = Table(table_data, colWidths=[200, 100])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    pdf.build(elements)

In [6]:
if __name__ == "__main__":
    generate_report_cards("student_scores.xlsx")

Report cards generated successfully in the 'report_cards' directory.
